In [2]:
#! pip install pandas
#! pip install openpyxl

### Este código genera un solo dataset de los dos casos json y csv

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from math import sqrt
import random
import re 

# --- CONFIGURACIÓN DE RUTAS ---
main_data_folder_path = 'datos/Usos bicimad/bicimad_2018/Consolidado' # Ruta de la carpeta con JSON/CSV de viajes
#C:\Users\ProjectSupport\Documents\David\Cursos\Master Data Science\TFM\Entrega TFM\datos\Usos bicimad\bicimad_2018
stations_folder_path = 'datos' # Ruta de la carpeta con el archivo de estaciones
stations_file_name = 'estaciones_bicimad_meteo_15_05.csv' # Usando el nombre de archivo que te funcionó
stations_file_path = os.path.join(stations_folder_path, stations_file_name)


# DEFINIR LA LISTA DE ARCHIVOS A PROCESAR Y SU ORDEN
files_to_process = [f for f in os.listdir(main_data_folder_path) if f.endswith('.json') or f.endswith('.csv')]

print("Iniciando la ejecución principal del procesador de datos unificado y optimizado.")

# --- CARGAR Y PREPROCESAR ARCHIVO DE ESTACIONES ---
df_estaciones = pd.DataFrame()
estaciones_lookup = {}
if os.path.exists(stations_file_path):
    try:
        df_estaciones = pd.read_csv(stations_file_path, encoding='utf-8', sep=';')
        df_estaciones.columns = df_estaciones.columns.str.strip() 
        
        print("\n--- Columnas del archivo de estaciones después de cargar y limpiar espacios: ---")
        print(df_estaciones.columns.tolist())
        print("-----------------------------------------------------------------------\n")

        station_column_mapping = {
            'id': 'station_id',
            'Longitud': 'longitude',
            'Latitud': 'latitude',
            'Número de Plazas': 'num_docks'
        }
        
        existing_cols_to_rename = {k: v for k, v in station_column_mapping.items() if k in df_estaciones.columns}
        df_estaciones = df_estaciones.rename(columns=existing_cols_to_rename)
        
        required_station_cols = ['station_id', 'longitude', 'latitude', 'num_docks']
        if not all(col in df_estaciones.columns for col in required_station_cols):
            missing_cols = [col for col in required_station_cols if col not in df_estaciones.columns]
            raise ValueError(f"Faltan columnas requeridas en el archivo de estaciones después de renombrar: {missing_cols}. Revise el mapeo de columnas y los nombres originales.")

        df_estaciones['longitude'] = pd.to_numeric(df_estaciones['longitude'], errors='coerce')
        df_estaciones['latitude'] = pd.to_numeric(df_estaciones['latitude'], errors='coerce')
        df_estaciones['station_id'] = pd.to_numeric(df_estaciones['station_id'], errors='coerce')
        df_estaciones['num_docks'] = pd.to_numeric(df_estaciones['num_docks'], errors='coerce')

        rows_before_dropna = len(df_estaciones)
        df_estaciones.dropna(subset=['station_id', 'longitude', 'latitude', 'num_docks'], inplace=True)
        rows_after_dropna = len(df_estaciones)

        if rows_before_dropna > rows_after_dropna:
            print(f"  Se eliminaron {rows_before_dropna - rows_after_dropna} filas del archivo de estaciones debido a valores nulos en columnas críticas.")
        
        if not df_estaciones.empty:
            df_estaciones['station_id'] = pd.to_numeric(df_estaciones['station_id'], errors='coerce').astype(pd.Int64Dtype())
            estaciones_lookup = df_estaciones.set_index('station_id')[['latitude', 'longitude', 'num_docks']].to_dict('index')
            print(f"Archivo de estaciones '{stations_file_name}' cargado exitosamente. {len(df_estaciones)} estaciones disponibles para referencia.")
        else:
            print(f"ADVERTENCIA: El archivo de estaciones '{stations_file_name}' terminó vacío después del preprocesamiento. 0 estaciones disponibles para referencia.")

    except Exception as e:
        print(f"ERROR: No se pudo cargar o preprocesar el archivo de estaciones '{stations_file_path}': {e}")
        df_estaciones = pd.DataFrame()
else:
    print(f"ADVERTENCIA: Archivo de estaciones no encontrado en '{stations_file_path}'. No se rellenarán IDs de estación faltantes.")

# Función para calcular la distancia euclidiana (aproximación para distancias cortas)
def euclidean_distance(lat1, lon1, lat2, lon2):
    return sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# Función para encontrar la estación más cercana
def find_nearest_station_id(target_lat, target_lon, stations_dict):
    if pd.isna(target_lat) or pd.isna(target_lon):
        return pd.NA, pd.NA

    min_dist = float('inf')
    nearest_station_id = pd.NA
    nearest_num_docks = pd.NA

    for station_id, data in stations_dict.items():
        s_lat = data['latitude']
        s_lon = data['longitude']
        s_num_docks = data['num_docks']

        if pd.isna(s_lat) or pd.isna(s_lon):
            continue

        dist = euclidean_distance(target_lat, target_lon, s_lat, s_lon)
        if dist < min_dist:
            min_dist = dist
            nearest_station_id = station_id
            nearest_num_docks = s_num_docks
    return nearest_station_id, nearest_num_docks


# Inicializar el DataFrame consolidado
all_processed_dfs = []

# --- MODIFICACIÓN: Columnas finales esperadas. Eliminadas 'ageRange' y 'user_day_code' ---
final_target_columns = [
    'unplug_hourTime', 'travel_time', 'idunplug_station', 'idplug_station',
    'idplug_base', 'idunplug_base',
    'unlock_station_name', 'lock_station_name'
]

print(f"\n--- INICIO DEL PROCESAMIENTO GENERAL ---")
print(f"Archivos a procesar (en este orden): {len(files_to_process)}")

for i, file in enumerate(files_to_process):
    file_path = os.path.join(main_data_folder_path, file)
    print(f"\n--- Procesando archivo: {file} ({i + 1}/{len(files_to_process)}) ---")

    df_temp = pd.DataFrame()

    try:
        if file.endswith('.json'):
            try:
                with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                    data = [json.loads(line) for line in f]
            except UnicodeDecodeError:
                with open(file_path, 'r', encoding='ISO-8859-1', errors='replace') as f:
                    data = [json.loads(line) for line in f]
            
            df_temp = pd.json_normalize(data)
            print(f"  Archivo JSON '{file}' cargado.")
            
            df_temp.columns = df_temp.columns.str.strip()

            # --- Procesamiento de unplug_hourTime para JSON ---
            col_time_unplug = 'unplug_hourTime'
            json_date_col_unplug = f'{col_time_unplug}.$date'

            if json_date_col_unplug in df_temp.columns:
                df_temp[col_time_unplug] = df_temp[json_date_col_unplug].apply(lambda x: x.get('$date') if isinstance(x, dict) else x)
                df_temp = df_temp.drop(columns=[json_date_col_unplug])
            elif col_time_unplug not in df_temp.columns:
                df_temp[col_time_unplug] = pd.NA 
            
            # Convertir unplug_hourTime a string para aplicar regex si es necesario
            df_temp[col_time_unplug] = df_temp[col_time_unplug].astype(str)

            # Limpiar la 'Z' y los offsets como +0100 o -0200, y milisegundos.
            # Esto convierte "2019-07-01T00:00:00Z" o "2019-07-01T00:00:00.123+0100" a "2019-07-01T00:00:00"
            df_temp[col_time_unplug] = df_temp[col_time_unplug].str.replace(r'\.\d+', '', regex=True) # Elimina milisegundos
            df_temp[col_time_unplug] = df_temp[col_time_unplug].str.replace(r'[Z]$', '', regex=True) # Elimina 'Z' al final
            df_temp[col_time_unplug] = df_temp[col_time_unplug].str.replace(r'[+-]\d{4}$', '', regex=True) # Elimina offsets como +0100

            # Convertir a datetime inmediatamente para liberar memoria antes de concatenar
            df_temp[col_time_unplug] = pd.to_datetime(df_temp[col_time_unplug], errors='coerce')


            # Extracción de coordenadas JSON (sin cambios)
            for prefix, lat_col, lon_col in [
                ('unplug_station.location.coordinates', 'temp_lat_unlock', 'temp_lon_unlock'),
                ('plug_station.location.coordinates', 'temp_lat_lock', 'temp_lon_lock'),
            ]:
                if prefix in df_temp.columns:
                    df_temp[lon_col] = df_temp[prefix].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 2 else pd.NA)
                    df_temp[lat_col] = df_temp[prefix].apply(lambda x: x[1] if isinstance(x, list) and len(x) == 2 else pd.NA)
                else:
                    if lat_col not in df_temp.columns: df_temp[lat_col] = pd.NA
                    if lon_col not in df_temp.columns: df_temp[lon_col] = pd.NA

            # IDs de estación JSON (sin cambios)
            if 'idunplug_station' not in df_temp.columns and 'unplug_station.id' in df_temp.columns:
                 df_temp['idunplug_station'] = pd.to_numeric(df_temp['unplug_station.id'], errors='coerce') 
            if 'idplug_station' not in df_temp.columns and 'plug_station.id' in df_temp.columns:
                 df_temp['idplug_station'] = pd.to_numeric(df_temp['plug_station.id'], errors='coerce')

            # Travel time JSON (sin cambios)
            if 'travel_time' in df_temp.columns:
                df_temp['travel_time'] = pd.to_numeric(df_temp['travel_time'], errors='coerce')
            
            # Columnas a eliminar específicas de JSON y las que quieres eliminar
            cols_to_drop_json_specific = [
                '_id.$oid', '_id', 
                'unplug_station.location.coordinates', 'plug_station.location.coordinates',
                'unplug_station.location.type', 'plug_station.location.type',
                'unplug_station.address', 'plug_station.address', 'track.features', 'track.type',
                'unplug_station.id', 'plug_station.id',
                'lock_date.$date' # Eliminada explícitamente si existe
            ]
            
            # ELIMINAR user_type, zip_code, _id_oid y lock_date si existen
            # --- MODIFICACIÓN: Eliminadas 'ageRange' y 'user_day_code' de esta lista para el JSON ---
            for col in ['_id_oid', 'user_type', 'zip_code', 'lock_date', 'ageRange', 'user_day_code']: 
                if col in df_temp.columns:
                    df_temp = df_temp.drop(columns=[col])

            cols_to_drop_json_from_df = [col for col in cols_to_drop_json_specific if col in df_temp.columns]
            if cols_to_drop_json_from_df:
                df_temp = df_temp.drop(columns=cols_to_drop_json_from_df)
            
            # Asegurar existencia de columnas de nombre de estación
            for col in ['unlock_station_name', 'lock_station_name']:
                if col not in df_temp.columns:
                    df_temp[col] = pd.NA


        elif file.endswith('.csv'):
            # --- LÓGICA DE PROCESAMIENTO CSV ---
            df_temp = pd.read_csv(file_path, encoding='ISO-8859-1', sep=';', on_bad_lines='skip', skip_blank_lines=True) 
            print(f"  Archivo CSV '{file}' cargado.")

            df_temp.columns = df_temp.columns.str.strip()
            df_temp.replace('', pd.NA, inplace=True)

            # Mapeo de columnas de CSV a nombres unificados
            csv_column_mapping = {
                'trip_minutes': 'travel_time',
                'unlock_date': 'unplug_hourTime', 
                'station_unlock': 'idunplug_station', 'dock_unlock': 'idunplug_base',      
                'station_lock': 'idplug_station', 'dock_lock': 'idplug_base',          
                'lock_date': 'lock_date_raw', # Leemos lock_date con un nombre temporal para el fallback
                'unlock_station_name': 'unlock_station_name',
                'lock_station_name': 'lock_station_name',

            }
            existing_csv_cols_to_rename = {k: v for k, v in csv_column_mapping.items() if k in df_temp.columns}
            df_temp = df_temp.rename(columns=existing_csv_cols_to_rename)
            
            # Convertir travel_time a numérico y a segundos (ya venía en minutos)
            if 'travel_time' in df_temp.columns:
                df_temp['travel_time'] = pd.to_numeric(df_temp['travel_time'], errors='coerce')
                df_temp.loc[df_temp['travel_time'].notna(), 'travel_time'] = df_temp.loc[df_temp['travel_time'].notna(), 'travel_time'] * 60
            else:
                df_temp['travel_time'] = pd.NA

            # --- Procesamiento de unplug_hourTime y lock_date para CSV ---
            # Asegurarse de que ambas columnas existen para la lógica de fallback
            if 'unplug_hourTime' not in df_temp.columns:
                df_temp['unplug_hourTime'] = pd.NaT # Inicializar con NaT para fechas
            if 'lock_date_raw' not in df_temp.columns:
                df_temp['lock_date_raw'] = pd.NaT

            # Primero, intentar convertir unplug_hourTime directamente
            df_temp['unplug_hourTime'] = pd.to_datetime(df_temp['unplug_hourTime'], errors='coerce')
            
            # Convertir lock_date_raw a datetime para el fallback
            df_temp['lock_date_raw'] = pd.to_datetime(df_temp['lock_date_raw'], errors='coerce')

            # Lógica de FALLBACK: Si unplug_hourTime es NaT, intentar calcularlo
            # desde lock_date_raw y travel_time
            mask_unplug_nan = df_temp['unplug_hourTime'].isna()
            mask_lock_valid = df_temp['lock_date_raw'].notna()
            mask_travel_valid = df_temp['travel_time'].notna() & (df_temp['travel_time'] >= 0) # Asegurar travel_time no negativo
            
            # Aplicar fallback solo a las filas que necesitan y tienen datos válidos
            fallback_mask = mask_unplug_nan & mask_lock_valid & mask_travel_valid
            
            if fallback_mask.any():
                print(f"  Aplicando lógica de fallback para {fallback_mask.sum()} filas con 'unplug_hourTime' nulo en '{file}'.")
                df_temp.loc[fallback_mask, 'unplug_hourTime'] = \
                    df_temp.loc[fallback_mask, 'lock_date_raw'] - \
                    pd.to_timedelta(df_temp.loc[fallback_mask, 'travel_time'], unit='s')
            
            # Eliminar la columna temporal lock_date_raw ya que ya no la necesitamos
            if 'lock_date_raw' in df_temp.columns:
                df_temp = df_temp.drop(columns=['lock_date_raw'])


            # --- LÓGICA PARA EXTRACCIÓN CONDICIONAL DE COORDENADAS CSV TEMPORALES ---
            def extract_coords_from_csv_geo_safe(geo_str_val):
                if pd.isna(geo_str_val) or not isinstance(geo_str_val, str):
                    return pd.NA, pd.NA
                try:
                    if geo_str_val.strip().startswith('(') and geo_str_val.strip().endswith(')'):
                        coords_clean = geo_str_val.strip('()').split(',')
                        if len(coords_clean) == 2:
                            lat = float(coords_clean[0].strip().replace(',', '.'))
                            lon = float(coords_clean[1].strip().replace(',', '.'))
                            return lat, lon
                    else:
                        json_str = geo_str_val.replace("'", '"')
                        geo_data = json.loads(json_str)
                        if 'coordinates' in geo_data and isinstance(geo_data['coordinates'], list) and len(geo_data['coordinates']) == 2:
                            lat = float(str(geo_data['coordinates'][1]).replace(',', '.'))
                            lon = float(str(geo_data['coordinates'][0]).replace(',', '.'))
                            return lat, lon
                except (json.JSONDecodeError, ValueError, TypeError, IndexError):
                    pass
                return pd.NA, pd.NA
            
            if 'temp_lat_unlock' not in df_temp.columns: df_temp['temp_lat_unlock'] = pd.NA
            if 'temp_lon_unlock' not in df_temp.columns: df_temp['temp_lon_unlock'] = pd.NA
            if 'temp_lat_lock' not in df_temp.columns: df_temp['temp_lat_lock'] = pd.NA
            if 'temp_lon_lock' not in df_temp.columns: df_temp['temp_lon_lock'] = pd.NA

            if 'geolocation_unlock' in df_temp.columns:
                df_temp['idunplug_station'] = pd.to_numeric(df_temp['idunplug_station'], errors='coerce') 
                mask_unlock_id_nan = df_temp['idunplug_station'].isna()
                if mask_unlock_id_nan.any():
                    df_temp.loc[mask_unlock_id_nan, ['temp_lat_unlock', 'temp_lon_unlock']] = \
                        df_temp.loc[mask_unlock_id_nan, 'geolocation_unlock'].apply(
                            lambda x: pd.Series(extract_coords_from_csv_geo_safe(x), dtype='object')).values
            
            if 'geolocation_lock' in df_temp.columns:
                df_temp['idplug_station'] = pd.to_numeric(df_temp['idplug_station'], errors='coerce') 
                mask_lock_id_nan = df_temp['idplug_station'].isna()
                if mask_lock_id_nan.any():
                    df_temp.loc[mask_lock_id_nan, ['temp_lat_lock', 'temp_lon_lock']] = \
                        df_temp.loc[mask_lock_id_nan, 'geolocation_lock'].apply(
                            lambda x: pd.Series(extract_coords_from_csv_geo_safe(x), dtype='object')).values
            
            # Eliminar filas completamente vacías (sin cambios)
            initial_rows = len(df_temp)
            df_temp.dropna(how='all', inplace=True)
            rows_dropped = initial_rows - len(df_temp)
            if rows_dropped > 0:
                print(f"  Se eliminaron {rows_dropped} filas completamente vacías de '{file}'.")

            # Columnas a eliminar específicas de CSV
            csv_columns_to_exclude_specific = {
                'fecha', 'idBike', 'fleet', 'address_unlock', 'locktype',
                'unlocktype', 'address_lock', 'geolocation',
                'geolocation_unlock', 'geolocation_lock'
            }
            
            # ELIMINAR user_type, zip_code, _id_oid si existen
            # --- MODIFICACIÓN: Eliminadas 'ageRange' y 'user_day_code' de esta lista para el CSV ---
            for col in ['_id_oid', 'user_type', 'zip_code', 'ageRange', 'user_day_code']: 
                if col in df_temp.columns:
                    df_temp = df_temp.drop(columns=[col])

            cols_to_drop_from_temp = [col for col in csv_columns_to_exclude_specific if col in df_temp.columns]
            if cols_to_drop_from_temp:
                df_temp = df_temp.drop(columns=cols_to_drop_from_temp)
            
            # Asegurar existencia de columnas de nombre de estación
            for col in ['unlock_station_name', 'lock_station_name']:
                if col not in df_temp.columns:
                    df_temp[col] = pd.NA

        else:
            print(f"  ADVERTENCIA: Archivo no reconocido (no .json ni .csv): '{file}'. Saltando.")
            continue

        # --- LÓGICA DE RELLENO DE IDS DE ESTACIÓN Y BASE ---
        if estaciones_lookup:
            # Relleno para unplug_station
            if 'idunplug_station' in df_temp.columns:
                df_temp['idunplug_station'] = pd.to_numeric(df_temp['idunplug_station'], errors='coerce')
                unplug_mask_nan = df_temp['idunplug_station'].isna()
                if unplug_mask_nan.any():
                    temp_coords_for_lookup = df_temp.loc[unplug_mask_nan, ['temp_lat_unlock', 'temp_lon_unlock']]
                    if not temp_coords_for_lookup.empty:
                        results_unplug = temp_coords_for_lookup.apply(
                            lambda row: pd.Series(find_nearest_station_id(row['temp_lat_unlock'], row['temp_lon_unlock'], estaciones_lookup), dtype='object'), 
                            axis=1
                        )
                        df_temp.loc[unplug_mask_nan, 'idunplug_station'] = results_unplug[0]
                        
                        base_unplug_mask_nan = df_temp.loc[unplug_mask_nan, 'idunplug_base'].isna() & df_temp.loc[unplug_mask_nan, 'idunplug_station'].notna()
                        if base_unplug_mask_nan.any():
                            docks_for_random = results_unplug[1].loc[base_unplug_mask_nan.index] 
                            valid_docks_mask = docks_for_random.notna() & (docks_for_random > 0)
                            if valid_docks_mask.any():
                                df_temp.loc[base_unplug_mask_nan.index[valid_docks_mask], 'idunplug_base'] = \
                                    [random.randint(1, int(d)) for d in docks_for_random[valid_docks_mask]]

            # Relleno para plug_station
            if 'idplug_station' in df_temp.columns:
                df_temp['idplug_station'] = pd.to_numeric(df_temp['idplug_station'], errors='coerce')
                plug_mask_nan = df_temp['idplug_station'].isna()
                if plug_mask_nan.any():
                    temp_coords_for_lookup = df_temp.loc[plug_mask_nan, ['temp_lat_lock', 'temp_lon_lock']]
                    if not temp_coords_for_lookup.empty:
                        results_plug = temp_coords_for_lookup.apply(
                            lambda row: pd.Series(find_nearest_station_id(row['temp_lat_lock'], row['temp_lon_lock'], estaciones_lookup), dtype='object'), 
                            axis=1
                        )
                        df_temp.loc[plug_mask_nan, 'idplug_station'] = results_plug[0]

                        base_plug_mask_nan = df_temp.loc[plug_mask_nan, 'idplug_base'].isna() & df_temp.loc[plug_mask_nan, 'idplug_station'].notna()
                        if base_plug_mask_nan.any():
                            docks_for_random = results_plug[1].loc[base_plug_mask_nan.index]
                            valid_docks_mask = docks_for_random.notna() & (docks_for_random > 0)
                            if valid_docks_mask.any():
                                df_temp.loc[base_plug_mask_nan.index[valid_docks_mask], 'idplug_base'] = \
                                    [random.randint(1, int(d)) for d in docks_for_random[valid_docks_mask]]


        # Eliminar las columnas de coordenadas temporales usadas para el relleno (sin cambios)
        cols_to_drop_temp_coords = ['temp_lat_unlock', 'temp_lon_unlock', 'temp_lat_lock', 'temp_lon_lock']
        cols_to_drop_from_temp_df = [col for col in cols_to_drop_temp_coords if col in df_temp.columns]
        if cols_to_drop_from_temp_df:
            df_temp = df_temp.drop(columns=cols_to_drop_from_temp_df)
        
        # Asegurarse de que el DataFrame temporal tenga todas las columnas objetivo antes de añadirlo
        # Las columnas que no existen en el df_temp pero están en final_target_columns serán rellenadas con NaN
        df_temp = df_temp.reindex(columns=final_target_columns)
        all_processed_dfs.append(df_temp)
        print(f"  Archivo '{file}' procesado y añadido a la lista para consolidación.")

    except FileNotFoundError:
        print(f"  ERROR: Archivo no encontrado: '{file_path}'. Saltando este archivo.")
    except (json.JSONDecodeError, UnicodeDecodeError, ValueError) as e:
        print(f"  ERROR: Problema de codificación o formato en '{file_path}': {e}. Saltando este archivo.")
    except pd.errors.EmptyDataError:
        print(f"  ADVERTENCIA: Archivo vacío o con solo encabezados: '{file_path}'. Saltando este archivo.")
    except Exception as e:
        print(f"  ERROR inesperado al procesar '{file}': {e}. Saltando este archivo.")

# --- CONSOLIDACIÓN FINAL ---
consolidated_df = pd.DataFrame()
if all_processed_dfs:
    print("\n--- Realizando la concatenación de todos los DataFrames procesados ---")
    consolidated_df = pd.concat(all_processed_dfs, ignore_index=True)
    print(f"Concatenación completada. Filas totales: {len(consolidated_df)}")
else:
    print("\nNo se procesó ningún archivo exitosamente. El DataFrame consolidado estará vacío.")

# --- CONVERSIÓN FINAL A ENTEROS (OPTIMIZADA) ---
if not consolidated_df.empty:
    print("\n--- Realizando conversiones finales a tipos enteros donde sea posible (optimizado) ---")
    # 'ageRange' ya no está en la lista de columnas a convertir
    integer_cols_final = ['travel_time', 'idplug_base', 'idplug_station', 'idunplug_base', 'idunplug_station'] 
    for col in integer_cols_final:
        if col in consolidated_df.columns:
            print(f"  Intentando convertir columna '{col}' (Dtype actual: {consolidated_df[col].dtype})...")
            
            current_series = pd.to_numeric(consolidated_df[col], errors='coerce')
            
            if pd.api.types.is_float_dtype(current_series):
                inf_mask = np.isinf(current_series)
                if inf_mask.any():
                    num_inf = inf_mask.sum()
                    current_series.loc[inf_mask] = np.nan
                    print(f"    Se reemplazaron {num_inf} valores infinitos por NaN en '{col}'.")
            
            try:
                temp_float_series = current_series.astype(float)
                # Solo redondear si hay parte decimal y no es NaN
                if (temp_float_series.notna() & (temp_float_series != temp_float_series.astype(int))).any():
                    temp_float_series = np.round(temp_float_series)

                consolidated_df[col] = temp_float_series.astype(pd.Int64Dtype())
                print(f"  Columna '{col}' convertida a {consolidated_df[col].dtype}.")

            except Exception as e:
                print(f"  ADVERTENCIA: No se pudo convertir '{col}' a Int64Dtype. Manteniendo el tipo numérico más general. Error: {e}")
                consolidated_df[col] = current_series 
                
    print("Conversiones a enteros completadas.")

# --- FORMATO FINAL DE unplug_hourTime a 'YYYY-MM-DDTHH:MM:SS' como STRING ---
# Este es el último paso, después del ordenamiento (que ahora no existe)
if 'unplug_hourTime' in consolidated_df.columns and pd.api.types.is_datetime64_any_dtype(consolidated_df['unplug_hourTime']):
    print("\n--- Formateando 'unplug_hourTime' a 'YYYY-MM-DDTHH:MM:SS' (string) ---")
    consolidated_df['unplug_hourTime'] = consolidated_df['unplug_hourTime'].dt.strftime('%Y-%m-%dT%H:%M:%S').replace({pd.NaT: pd.NA})
    print("Formato final aplicado a 'unplug_hourTime'.")


print("\n--- FIN DEL PROCESAMIENTO GENERAL ---")

# --- Mostrar el resultado final ---
if 'consolidated_df' in locals() and not consolidated_df.empty:
    print("\n--- RESULTADO FINAL: DataFrame Consolidado ---")
    print("Primeras 5 filas:") # Ya no se asume que está ordenado aquí
    print(consolidated_df.head())
    print(f"\nDimensiones del DataFrame consolidado: {consolidated_df.shape}")
    print("\nTipos de datos del DataFrame consolidado (después de conversiones finales):")
    print(consolidated_df.dtypes) 
    print("\nColumnas del DataFrame consolidado:")
    print(consolidated_df.columns.tolist())
    
    if 'unplug_hourTime' in consolidated_df.columns: 
        print(f"\nLa columna 'unplug_hourTime' ahora es de tipo: {consolidated_df['unplug_hourTime'].dtype}")
    
    if 'unplug_hourTime' in consolidated_df.columns:
        num_na_final = consolidated_df['unplug_hourTime'].isna().sum()
        if num_na_final > 0:
            print(f"\nNúmero de filas con 'unplug_hourTime' nulo/inválido (NaN/pd.NA) en el DataFrame final: {num_na_final}")
        else:
            print(f"\n¡Excelente! No se encontraron valores nulos/inválidos en 'unplug_hourTime' en el DataFrame final.")
else:
    print("\nEl DataFrame consolidado final está vacío. Verifica la ruta y los archivos.")

Iniciando la ejecución principal del procesador de datos unificado y optimizado.

--- Columnas del archivo de estaciones después de cargar y limpiar espacios: ---
['id', 'Número', 'Gis_X', 'Gis_Y', 'Fecha de Alta', 'Distrito', 'Barrio', 'Calle', 'Nº Finca', 'Tipo de Reserva', 'Número de Plazas', 'Longitud', 'Latitud', 'Direccion', 'E. Temp. Código 1', 'E. Temp. Código 2', 'E. Temp. Código 3', 'E. Prec. Código 1', 'E. Prec. Código 2']
-----------------------------------------------------------------------

Archivo de estaciones 'estaciones_bicimad_meteo_15_05.csv' cargado exitosamente. 271 estaciones disponibles para referencia.

--- INICIO DEL PROCESAMIENTO GENERAL ---
Archivos a procesar (en este orden): 12

--- Procesando archivo: 201801_Usage_Bicimad.json (1/12) ---
  Archivo JSON '201801_Usage_Bicimad.json' cargado.
  Archivo '201801_Usage_Bicimad.json' procesado y añadido a la lista para consolidación.

--- Procesando archivo: 201802_Usage_Bicimad.json (2/12) ---
  Archivo JSON '2

In [6]:
# contar numero de filas del df
print(f"\nNúmero total de filas en el DataFrame consolidado: {len(consolidated_df)}")


Número total de filas en el DataFrame consolidado: 3678086


In [7]:
# Ver las primeras filas para validar
consolidated_df.head()

,unplug_hourTime,travel_time,idunplug_station,idplug_station,idplug_base,idunplug_base,unlock_station_name,lock_station_name
0,2018-01-01T00:00:00,284,6,7,1,14,NaN,NaN
1,2018-01-01T00:00:00,666,24,117,4,21,NaN,NaN
2,2018-01-01T00:00:00,662,24,117,19,19,NaN,NaN
3,2018-01-01T00:00:00,708,82,110,1,2,NaN,NaN
4,2018-01-01T00:00:00,171,169,58,5,3,NaN,NaN


In [8]:
#guardar el DataFrame como un archivo CSV
consolidated_df.to_csv('bdcompletabicimadcsvprueba.csv', index=False)

In [ ]:
# Lista con el orden deseado de las columnas
columnas_orden_deseado = ['idunplug_station', 'idplug_station', 'travel_time', 'idplug_base',
                            'idunplug_base','unplug_hourTime']
consolidated_df = consolidated_df[columnas_orden_deseado]

columnas_eliminar = ["unlock_station_name", "lock_station_name"]
consolidated_df.drop(columns=columnas_eliminar, inplace=True)


In [13]:
# # Optimizado: convertir varias columnas a entero de forma segura
cols_to_int = ['idplug_base','idplug_station']
consolidated_df[cols_to_int] = consolidated_df[cols_to_int].apply(pd.to_numeric, errors='coerce').astype('Int64')
consolidated_df.head()

,idunplug_station,idplug_station,travel_time,idplug_base,idunplug_base,unplug_hourTime
0,6,7,284,1,14,2018-01-01T00:00:00
1,24,117,666,4,21,2018-01-01T00:00:00
2,24,117,662,19,19,2018-01-01T00:00:00
3,82,110,708,1,2,2018-01-01T00:00:00
4,169,58,171,5,3,2018-01-01T00:00:00


In [14]:
# contar nulos de todas las columnas
nulos_por_columna_csv = consolidated_df.isnull().sum()
print(nulos_por_columna_csv)

idunplug_station    0
idplug_station      0
travel_time         0
idplug_base         0
idunplug_base       0
unplug_hourTime     0
dtype: int64


In [15]:
#guardar el DataFrame como un archivo CSV
consolidated_df.to_csv('bdcompleta_usos_bicimad_2018.csv', index=False)